In [1]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv("train.csv")
passager = pd.read_csv("gender_submission.csv")
print(train["Survived"].value_counts())
train.head()

0    549
1    342
Name: Survived, dtype: int64


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Functions creation

In [3]:
#creating cleaning functions 
def Fillage(dataset): 
    average = dataset.Age.mean()
    dataset.Age = dataset.Age.fillna(average)#Fill empty of Age with mean()
    return
def children(dataset): #creating a 'childre' feature
    dataset["Child"] = float('NaN')
    dataset['Child'][dataset['Age'] < 18]=1
    dataset['Child'][dataset['Age'] >= 18] =0
    return dataset
    
def sexCat(dataset): #creating categories for sex
    dataset.loc[train['Sex'] == 'male', 'Sex']=0
    dataset.loc[train['Sex'] == 'female', 'Sex']=1
    dataset.Sex = train.Sex.astype('category')

def embarcation(dataset):
    dataset["Embarked"] = dataset["Embarked"].fillna(0)
    dataset.loc[dataset['Embarked'] == 'S', 'Embarked']=0
    dataset.loc[dataset['Embarked'] == 'C', 'Embarked']=1
    dataset.loc[dataset['Embarked'] == 'Q', 'Embarked']=2
    dataset.Embarked = dataset.Embarked.astype('category')
    
def title(dataset): # create classes with title and transfor in dummies
    dataset['title']= dataset['Name'].apply(lambda x : x[x.index(","):x.index(".")][2:])
    return dataset
    
def cabins(dataset): #fill empty cabin cells
    dataset.Cabin = dataset.Cabin.fillna('noCab')
    return dataset


def cabtypes(dataset):
    dataset['CabType']= dataset['Cabin'].apply(lambda x:str(x)[0]) 
    




# Cleaning the train set

In [4]:
#Data Cleaning and feature engineering
Fillage(train)
children(train)
sexCat(train)
embarcation(train)
title(train)
cabins(train)
cabtypes(train)

titre = pd.crosstab(train["Survived"], train["title"], normalize=False)


#regrouper les catégories
#Creer catégories Avec dictionnaire
dic={'Lady':'survit', 'Mlle':'survit','Mme' : 'survit', 'Ms' : 'survit', 'Sir': 'survit', 'the Countess':'survit', 'Capt':'mort', 'Don':'mort', 'Jonkheer':'mort', 'Rev':'mort', 'Col':'moitie', 'Major':'moitie', 'Dr':'moitie' }

for key,value in dic.items():
    train.loc[train['title']==key, 'title']=value

titre = pd.crosstab(train["Survived"], train["title"], normalize=False)


dummies = pd.get_dummies(train['title'])
train = train.drop('title', axis=1)
train = pd.concat([train, dummies], axis=1)

cabins(train)
cabtypes(train)


train.loc[train['CabType'] == 'E', 'CabType']='D'


cabines = pd.crosstab(train['CabType'], train['Survived'])




dummies2 = pd.get_dummies(train['CabType'])
train = train.drop('CabType', axis=1)
train = pd.concat([train, dummies2], axis=1)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 28 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null category
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          891 non-null object
Embarked       891 non-null category
Child          891 non-null float64
Master         891 non-null uint8
Miss           891 non-null uint8
Mr             891 non-null uint8
Mrs            891 non-null uint8
moitie         891 non-null uint8
mort           891 non-null uint8
survit         891 non-null uint8
A              891 non-null uint8
B              891 non-null uint8
C              891 non-null uint8
D              891 non-null uint8
F              891 non-null uint8
G              891 non-null ui

C:\Users\Dimitri\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Dimitri\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Test Set

In [5]:
#Load test set
test = pd.read_csv("test.csv")

#Data Cleaning and feature engineering
Fillage(test)
children(test)
sexCat(test)
embarcation(test)
cabins(train)

test['title']= test['Name'].apply(lambda x : x[x.index(","):x.index(".")][2:])

#regrouper les catégories
#regrouper les catégories
#Creer catégories Avec dictionnaire
dic={'Lady':'survit', 'Mlle':'survit','Mme' : 'survit', 'Ms' : 'survit', 'Sir': 'survit', 'the Countess':'survit', 'Capt':'mort', 'Don':'mort', 'Jonkheer':'mort', 'Rev':'mort', 'Col':'moitie', 'Major':'moitie', 'Dr':'moitie' }

for key,value in dic.items():
    test.loc[test['title']==key, 'title']=value


dummies = pd.get_dummies(test['title'])
test = test.drop('title', axis=1)
test = pd.concat([test, dummies], axis=1)


empty = {'Cabin': 'noCab'}
test = test.fillna(value=empty)

average2 = test.Fare.mean()
test.Fare = test.Fare.fillna(average2)#Fill empty of Fare with mean()

cabtypes(test)


test.loc[test['CabType'] == 'E', 'CabType']='D'



dummies2 = pd.get_dummies(test['CabType'])
test = test.drop('CabType', axis=1)
test = pd.concat([test, dummies2], axis=1)
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 27 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null category
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          418 non-null object
Embarked       418 non-null category
Child          418 non-null float64
Dona           418 non-null uint8
Master         418 non-null uint8
Miss           418 non-null uint8
Mr             418 non-null uint8
Mrs            418 non-null uint8
moitie         418 non-null uint8
mort           418 non-null uint8
survit         418 non-null uint8
A              418 non-null uint8
B              418 non-null uint8
C              418 non-null uint8
D              418 non-null uint8
F              418 non-null uint8
G              418 non-null ui

C:\Users\Dimitri\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Dimitri\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Creating the model (random forest)

In [6]:
#create X and Y  (training)
y = train.Survived
x = train[['Pclass','Sex', 'Age', 'SibSp',  'Parch', 'Embarked', 'Fare', 'Child', 'Miss', 'Mr','Mrs','Master', 'moitie', 'mort', 'survit' , "A",  "B",  "C",  "D", "F", "G", "n"]]

y = np.ravel(y)

# Building and fitting my_forest
forest = RandomForestClassifier(max_depth = 10, min_samples_split=2, n_estimators = 100, random_state = 1)
my_forest = forest.fit(x, y)

# Print the score of the fitted random forest
print(my_forest.score(x, y))

# Compute predictions on our test set features then print the length of the prediction vector
test_features = test[['Pclass','Sex', 'Age', 'SibSp',  'Parch', 'Embarked', 'Fare', 'Child', 'Miss', 'Mr','Mrs','Master', 'moitie', 'mort', 'survit' , "A",  "B",  "C",  "D", "F", "G", "n"]]

pred_forest = my_forest.predict(test_features)
print(len(pred_forest))



0.931537598204
418


In [7]:
PassengerId =np.array(test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(pred_forest, PassengerId, columns = ["Survived"])

my_solution.to_csv("sol8.csv", index_label = ["PassengerId"])